## 4.1. Preparing building mask

In [ ]:
from osgeo import gdal, ogr, osr # Import osr for spatial reference operations

# --- User Parameters ---
# Change these paths and parameters to suit your data
raster_path = r"C:\Users\HojungYu\Downloads\tile_108_sliced_inputs\template_tile_108.tif"
vector_path = r"C:\Users\HojungYu\Downloads\tile_108_sliced_inputs\buildings_tile_108.shp"
out_raster = r"C:\Users\HojungYu\Downloads\tile_108_sliced_inputs\buildings.tif"
pixel_size = 1.0 # Desired resolution of the output raster in CRS units

# --- 1. Open the REFERENCE raster to get its extent, geotransform, and projection ---
# We use gdal.Open() for raster files.
ref_raster_ds = gdal.Open(raster_path)
if ref_raster_ds is None:
    raise RuntimeError(f"Error: Could not open reference raster file at {raster_path}")

# Get the geotransform (contains origin and pixel size) and projection from the reference raster.
# A geotransform is a tuple of 6 values: (top-left X, pixel width, row rotation, top-left Y, column rotation, pixel height)
gt = ref_raster_ds.GetGeoTransform()
prj = ref_raster_ds.GetProjection()

# Calculate the precise extent (minX, maxX, minY, maxY) of the reference raster.
# gt[0] is top-left X (minX)
# gt[3] is top-left Y (maxY)
# gt[1] is pixel width (E-W resolution)
# gt[5] is pixel height (N-S resolution, typically negative for GeoTIFFs as Y decreases with row index)
minx_ref = gt[0]
maxy_ref = gt[3]
maxx_ref = minx_ref + ref_raster_ds.RasterXSize * gt[1]
miny_ref = maxy_ref + ref_raster_ds.RasterYSize * gt[5] # gt[5] is usually negative

# It's good practice to close the dataset once we've extracted necessary information.
ref_raster_ds = None

# --- 2. Open the VECTOR dataset ---
# We use ogr.Open() for vector files.
src_ds = ogr.Open(vector_path)
if src_ds is None:
    raise RuntimeError(f"Error: Could not open vector dataset file at {vector_path}")
layer = src_ds.GetLayer(0) # Get the first layer from the vector dataset

# --- 3. Compute output raster dimensions ---
# The output raster dimensions are based on the full extent of the reference raster
# and the desired output 'pixel_size'.
# We use round() before converting to int to ensure correct rounding for pixel counts.
x_res = int(round((maxx_ref - minx_ref) / pixel_size))
y_res = int(round((maxy_ref - miny_ref) / pixel_size)) # Note: (maxy_ref - miny_ref) will be positive

# --- 4. Create the output raster file ---
# Get the GeoTIFF driver to create the output raster.
drv = gdal.GetDriverByName("GTiff")
if drv is None:
    raise RuntimeError("Error: GTiff driver not found. Is GDAL installed correctly?")

# Create the new raster dataset with the calculated dimensions, 1 band, and byte data type.
# gdal.GDT_Byte is suitable for simple masks (0 or 1 values).
out_ds = drv.Create(out_raster, x_res, y_res, 1, gdal.GDT_Byte)
if out_ds is None:
    raise RuntimeError(f"Error: Could not create output raster file at {out_raster}")

# Set the geotransform for the output raster.
# We use the minX and maxY from the reference raster's extent,
# the desired pixel_size, and preserve the skew (rotation) terms from the original geotransform (gt[2], gt[4]).
out_ds.SetGeoTransform((minx_ref, pixel_size, gt[2], maxy_ref, gt[4], -pixel_size))

# Set the projection for the output raster to match the reference raster. This is crucial for spatial accuracy.
out_ds.SetProjection(prj)

# Get the first band of the output raster to work with it.
band = out_ds.GetRasterBand(1)

# Initialize all pixels in the band to 1.
# This means areas with "no building" will have a value of 1.
band.Fill(1)

# Set 1 as the NoData value. This indicates that pixels with value 1 (no building)
# should be treated as NoData by GIS software. If you want 0 to be NoData, change this.
band.SetNoDataValue(0)

# --- 5. Burn (rasterize) the vector features into the output raster ---
# gdal.RasterizeLayer burns the features of a vector layer into a raster dataset.
# We will burn a value of 0 wherever a building footprint exists.
# The 'options' parameter can include SQL-like WHERE clauses to filter features.
# Here, we assume 'building_footprint' is a field in your shapefile used to identify buildings.
rasterize_options = ["WHERE building_footprint != 0"]

gdal.RasterizeLayer(
    out_ds,           # The output raster dataset to burn into
    [1],              # List of band indices to burn into (here, just band 1)
    layer,            # The OGR layer (your building footprints) to rasterize
    burn_values=[0],  # The value to burn into the raster pixels for features that match the options
    options=rasterize_options # Options for rasterization, including WHERE clause
)

# --- 6. Clean up ---
# Flush the cache to ensure all data is written to the disk.
band.FlushCache()

# Close the datasets to release file locks and save changes.
out_ds = None
src_ds = None

print(f"Raster mask successfully created and written to {out_raster}")



Raster mask successfully created and written to C:\Users\HojungYu\Downloads\tile_108_sliced_inputs\building_mask.tif
